# RNA-Seq Analysis Training Demo (Snakemake)

## Overview

This short tutorial demonstrates how to run an RNA-Seq workflow using a prokaryotic data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression. This tutorial uses a popular workflow manager called 'snakemake'. More information on snakemake can be found <a href="https://snakemake.readthedocs.io/en/stable/">here</a>. Running the code in this tutorial will take approximately 12 minutes.

![RNA-Seq workflow](images/rnaseq-workflow.png)

### STEP 1: Install mambaforge and snakemake
First install mambaforge.

We will use it to install snakemake, as well as create a snakemake environment using mambaforge.

In [1]:
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
!bash Mambaforge-$(uname)-$(uname -m).sh -b -p $HOME/mambaforge
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda snakemake
!cp /home/jupyter/mambaforge/bin/mamba /opt/conda/bin

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 88.1M  100 88.1M    0     0  21.8M      0  0:00:04  0:00:04 --:--:-- 25.7M
ERROR: File or directory already exists: '/home/jupyter/mambaforge'
If you want to update an existing installation, use the -u option.

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔█

### STEP 2: Create directories that will be used in our analysis

In [2]:
!echo $PWD
!mkdir -p data
!mkdir -p data/raw_fastq
!mkdir -p data/trimmed
!mkdir -p data/reference
!mkdir -p data/fastqc
!mkdir -p envs

/home/jupyter/rnaseq-myco-notebook


### STEP 3: Copy FASTQ Files
In order for this tutorial to run quickly, we will only analyze 50,000 reads from a sample from both sample groupsinstead of analyzing all the reads from all six samples. These files have been posted on a Google Storage Bucket that we made publicly accessible.


In [3]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349122_1.fastq --output data/raw_fastq/SRR13349122_1.fastq
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349122_2.fastq --output data/raw_fastq/SRR13349122_2.fastq
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349128_1.fastq --output data/raw_fastq/SRR13349128_1.fastq
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349128_2.fastq --output data/raw_fastq/SRR13349128_2.fastq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8452k  100 8452k    0     0   161M      0 --:--:-- --:--:-- --:--:--  165M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8452k  100 8452k    0     0   180M      0 --:--:-- --:--:-- --:--:--  183M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8452k  100 8452k    0     0   162M      0 --:--:-- --:--:-- --:--:--  165M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8452k  100 8452k    0     0   157M      0 --:--:-- --:--:-- --:--:--  158M


### STEP 4: Copy reference transcriptome files that will be used by Salmon
Salmon is a tool that aligns RNA-Seq reads to a set of transcripts rather than the entire genome.

In [4]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_transcripts.fasta --output data/reference/M_chelonae_transcripts.fasta
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/data/reference/decoys.txt --output data/reference/decoys.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9599k  100 9599k    0     0   140M      0 --:--:-- --:--:-- --:--:--  142M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0   1032      0 --:--:-- --:--:-- --:--:--  1076


### STEP 5: Copy data file for Trimmomatic

In [5]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa --output TruSeq3-PE.fa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    95  100    95    0     0   7065      0 --:--:-- --:--:-- --:--:--  7307


### STEP 6: Download data and config files that will be used in our snakemake environment

Next download config files for our snakemake environment, as well as data files which we will analyze.

In [6]:
# Copy config and data files

!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/config.yaml --output config.yaml
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/snakefile --output snakefile
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/envs/fastqc.yaml --output envs/fastqc.yaml
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/envs/trimmomatic.yaml --output envs/trimmomatic.yaml
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/envs/salmon.yaml --output envs/salmon.yaml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    67  100    67    0     0   5107      0 --:--:-- --:--:-- --:--:--  5153
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3561  100  3561    0     0   270k      0 --:--:-- --:--:-- --:--:--  289k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   106  100   106    0     0   7378      0 --:--:-- --:--:-- --:--:--  7571
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   106  100   106    0     0   8086      0 --:--:-- --:--:-- --:--:--  8153
  % Total    % Received % Xferd  Average Speed   Tim

#### Explanation of config files

Snakemake is unique in that it uses config files to manage workflows in the form of 'yaml' files, as well as a 'snakefile'.

Below is a brief example of some of the yaml config files:

In [7]:
!printf "The config.yaml file contains our sample names:\n\n Config.yaml\n"
!cat config.yaml
!printf "\n\nThe env folder contains information pertaining to packages to be used in the environment, \nas well as their version, for example, here is the 'envs/fastqc.yaml' file:\n\n Fastqc.Yaml\n"
!cat envs/fastqc.yaml

The config.yaml file contains our sample names:

 Config.yaml
samples:
    SRR13349122: SRR13349122
    SRR13349128: SRR13349128


The env folder contains information pertaining to packages to be used in the environment, 
as well as their version, for example, here is the 'envs/fastqc.yaml' file:

 Fastqc.Yaml
channels:
  - bioconda
  - conda-forge
  - defaults
dependencies:
  - fastqc ==0.11.9
  - multiqc ==1.12



### STEP 7: Run snakemake on our snakefile

Aside from the .yaml config files which information about software, dependencies, and versions -- snakemake uses a snakefile which contains information about a workflow.

This can be a powerful tool as it allows one to operate and think in terms of workflows instead of individual steps. 

Feel free to open the snakefile to look at it further. It is composed of 'rules' we have created.

Snakefiles work largely based on inputs. For a given input, there is an associated 'rule' that runs.

Snakefiles may take a while to get the idea of what's going on, but in simplest terms here we take an input of .fastq files, and based on the snakefile rules we created, those fastq files are run through the entire workflow of tutorial one.


In [8]:
!conda config --set channel_priority strict
!snakemake --cores --use-conda --forceall

{'SRR13349122': 'SRR13349122', 'SRR13349128': 'SRR13349128'}
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 4
Rules claiming more threads will be scaled down.
Job stats:
job                   count    min threads    max threads
------------------  -------  -------------  -------------
all                       1              1              1
fastqc_trimmed            1              4              4
multiqc_trimmed           1              1              1
salmon_index              1              4              4
salmon_quant_reads        2              2              2
trimmomatic_pe_fq         2              4              4
total                     8              1              4

Select jobs to execute...

[Wed Jun  1 15:37:46 2022]
rule trimmomatic_pe_fq:
    input: data/raw_fastq/SRR13349128_1.fastq, data/raw_fastq/SRR13349128_2.fastq
    output: data/trimmed/SRR13349128_trimmed_1.fastq, data/trimmed/SRR13349128_trimmed_2.fastq, data/trimmed/SRR13349128_trimme

### STEP 8: Report the top 10 most highly expressed genes in the samples.

Top 10 most highly expressed genes in the wild-type sample. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`


In [9]:
!sort -nrk 5,5 data/quants/SRR13349122_quant/quant.sf | head -10


BB28_RS20665	1293	1043.000	5447.071698	55.000
BB28_RS03075	1626	1376.000	3077.869008	41.000
BB28_RS07370	9255	9005.000	424.426717	37.000
BB28_RS20690	10377	10127.000	367.203150	36.000
BB28_RS19405	3948	3698.000	1005.588509	36.000
BB28_RS18585	1305	1055.000	2741.512828	28.000
BB28_RS20685	7731	7481.000	372.811085	27.000
BB28_RS19310	1194	944.000	2626.176789	24.000
BB28_RS22260	1836	1586.000	1497.991546	23.000
BB28_RS09805	1437	1187.000	2001.528725	23.000
sort: write failed: 'standard output': Broken pipe
sort: write error


Top 10 most highly expressed genes in the double lysogen sample.


In [10]:
!sort -nrk 5,5 data/quants/SRR13349128_quant/quant.sf | head -10


BB28_RS20665	1293	1043.000	16640.051824	107.000
BB28_RS18585	1305	1055.000	5381.096618	35.000
BB28_RS13330	2790	2540.000	2107.343957	33.000
BB28_RS14905	972	722.000	6065.711819	27.000
BB28_RS22260	1836	1586.000	1943.146845	19.000
BB28_RS18315	1767	1517.000	2031.529926	19.000
BB28_RS20685	7731	7481.000	368.590781	17.000
BB28_RS20690	10377	10127.000	240.251247	15.000
BB28_RS03075	1626	1376.000	1768.186333	15.000
BB28_RS11085	1278	1028.000	2208.971570	14.000
sort: write failed: 'standard output': Broken pipe
sort: write error


### STEP 9: Report the expression of a putative acyl-ACP desaturase (BB28_RS16545) that was downregulated in the double lysogen relative to wild-type
A acyl-transferase was reported to be downregulated in the double lysogen as shown in the table of the top 20 upregulated and downregulated genes from the paper describing the study.
![RNA-Seq workflow](images/table-cushman.png)

Use `grep` to report the expression in the wild-type sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [11]:
!grep 'BB28_RS16545' data/quants/SRR13349122_quant/quant.sf


BB28_RS16545	987	737.000	560.631139	4.000


Use `grep` to report the expression in the double lysogen sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [12]:
!grep 'BB28_RS16545' data/quants/SRR13349128_quant/quant.sf


BB28_RS16545	987	737.000	220.083619	1.000


## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as the standard short or extended tutorials.


[Workflow One:](Tutorial_1.ipynb) A short introduction to downloading and mapping sequences to a transcriptome using Trimmomatic and Salmon. Here is a link to the YouTube video demonstrating the tutorial: <https://www.youtube.com/watch?v=NG1U7D4l31o&t=26s>.

[Workflow One (Extended):](Tutorial_1B_Extended.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset, and includes elaboration such as using SRA tools for sequence downloading, and examples of running batches of fastq files through the pipeline. This workflow may take around an hour to run.

[Workflow One (Using Snakemake):](Tutorial_2_Snakemake.ipynb) Using snakemake to run workflow one.

[Workflow Two (DEG Analysis):](Tutorial_3_DEG_Analysis.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.


![RNA-Seq workflow](images/RNA-Seq_Notebook_Homepage.png)